## Model training and tuning for the energy sector

In [1]:
import pandas as pd
import numpy as np

import time

from darts import TimeSeries, concatenate
from darts.utils.model_selection import train_test_split
from darts.models.forecasting.xgboost import XGBModel
from darts.dataprocessing.transformers.scaler import Scaler
from darts.metrics.metrics import mape, mae
from darts.models.forecasting.prophet_model import Prophet
from darts.models.forecasting.sf_auto_theta import AutoTheta
from darts.models.forecasting.linear_regression_model import LinearRegressionModel
from darts.dataprocessing.transformers import StaticCovariatesTransformer
from darts.models.forecasting.sf_auto_arima import AutoARIMA
from darts.dataprocessing.transformers.boxcox import BoxCox

Importing plotly failed. Interactive plots will not work.
d:\investing-bot\.venv\Lib\site-packages\fs\__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__("pkg_resources").declare_namespace(__name__)  # type: ignore


In [2]:
df = pd.read_csv('../../data/processed/sp500_all_companies_cleaned_with_sector.csv')
df['date'] = pd.to_datetime(df['date'])

df_communication = df[df['GICS Sector'] == 'Communication Services'].sort_values(by=['ticker', 'date']).reset_index(drop=True)

In [3]:
df['GICS Sector'].value_counts()

GICS Sector
Industrials               193459
Financials                190485
Information Technology    177149
Health Care               151348
Consumer Discretionary    130185
Consumer Staples           96649
Real Estate                79757
Utilities                  78786
Materials                  63394
Energy                     58523
Communication Services     58017
Name: count, dtype: int64

In [4]:
df_communication['ticker'].unique()

array(['CHTR', 'CMCSA', 'DIS', 'EA', 'FOX', 'FOXA', 'GOOG', 'GOOGL',
       'IPG', 'LYV', 'META', 'MTCH', 'NFLX', 'NWS', 'NWSA', 'OMC', 'PARA',
       'T', 'TKO', 'TMUS', 'TTWO', 'VZ', 'WBD'], dtype=object)

In [5]:
data = {}
static_covs = pd.DataFrame({'ticker': df_communication['ticker'].unique()})

for ticker, group in df_communication.groupby('ticker'):
    series = TimeSeries.from_dataframe(
        group,
        time_col='date',
        value_cols=['close', 'high', 'low', 'open', 'volume'],
        freq='B',
        static_covariates=pd.Series(ticker)
    )
    data[ticker] = series

In [6]:
# encode each ticker as a static covariate
transformer = StaticCovariatesTransformer()
ts = [d for d in data.values()]
transformer.fit(ts)

for ticker, series in data.items():
    data[ticker] = transformer.transform(series)

In [7]:
train = {}
test = {}

for ticker, series in data.items():
    train[ticker], test[ticker] = train_test_split(series, test_size=0.1)
    last_train = train[ticker][-1:]
    test_with_context = last_train.append(test[ticker]) # append last point of train to test as our forecast requires it
    test[ticker] = test_with_context

## Gradient Boosting

### Train a local model first

In [8]:
models = {}

for ticker in data.keys():
    print(f"Training model for {ticker}")
    start = time.time()
    model = XGBModel(
        lags=15,
        lags_past_covariates=1,
        lags_future_covariates=[0],
        output_chunk_length=1,
        random_state=42,
        add_encoders={
            'datetime_attribute': {
                'future': ['month', 'day', 'dayofweek', 'dayofyear', 'quarter', 'year'] # automatically generate future covariates from time series
            }
        }
    )

    model.fit(
        train[ticker]['close'],
        past_covariates=train[ticker][['high', 'low', 'volume', 'open']],
    )

    end = time.time()
    print(f"Model for {ticker} trained in {end - start:.2f} seconds")
    print("*" * 50)

    models[ticker] = model

Training model for CHTR
Model for CHTR trained in 0.31 seconds
**************************************************
Training model for CMCSA
Model for CMCSA trained in 0.37 seconds
**************************************************
Training model for DIS
Model for DIS trained in 0.35 seconds
**************************************************
Training model for EA
Model for EA trained in 0.34 seconds
**************************************************
Training model for FOX
Model for FOX trained in 0.29 seconds
**************************************************
Training model for FOXA
Model for FOXA trained in 0.31 seconds
**************************************************
Training model for GOOG
Model for GOOG trained in 0.32 seconds
**************************************************
Training model for GOOGL
Model for GOOGL trained in 0.30 seconds
**************************************************
Training model for IPG
Model for IPG trained in 0.40 seconds
*******************************

In [9]:
mapes = {}
maes = {}

for ticker in data.keys():
    y_test = test[ticker]['close']
    y_pred = models[ticker].predict(
        n=len(y_test),
        past_covariates=test[ticker][['high', 'low', 'volume', 'open']],
        show_warnings=False
    )
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)
    mapes[ticker] = mape_
    maes[ticker] = mae_
    print(f"Ticker: {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Ticker: CHTR, MAPE: 5.18, MAE: 18.22
**************************************************
Ticker: CMCSA, MAPE: 3.25, MAE: 1.25
**************************************************
Ticker: DIS, MAPE: 4.56, MAE: 4.29
**************************************************
Ticker: EA, MAPE: 11.27, MAE: 16.71
**************************************************
Ticker: FOX, MAPE: 21.25, MAE: 10.07
**************************************************
Ticker: FOXA, MAPE: 17.39, MAE: 8.80
**************************************************
Ticker: GOOG, MAPE: 8.53, MAE: 15.56
**************************************************
Ticker: GOOGL, MAPE: 7.91, MAE: 14.37
**************************************************
Ticker: IPG, MAPE: 2.37, MAE: 0.67
**************************************************
Ticker: LYV, MAPE: 8.33, MAE: 10.67
**************************************************
Ticker: META, MAPE: 16.98, MAE: 100.50
**************************************************
Ticker: MTCH, MAPE: 21.52, MAE: 6.9

In [10]:
np.mean(list(mapes.values())), np.mean(list(maes.values()))

(11.528449102259483, 23.301251696859143)

### Global model

In [11]:
print(f"Training global model")
start = time.time()
model = XGBModel(
    lags=15,
    lags_past_covariates=1,
    lags_future_covariates=[0],
    output_chunk_length=1,
    random_state=42,
    add_encoders={
        'datetime_attribute': {
            'future': ['month', 'day', 'dayofweek', 'dayofyear', 'quarter', 'year'] # automatically generate future covariates from time series
        }
    },
    use_static_covariates=True,
    n_estimators=1000,
    eta=0.01,
    max_depth=8,
    subsample=0.8
)

model.fit(
    [ts['close'] for ts in train.values()],
    past_covariates=[ts[['high', 'low', 'volume', 'open']] for ts in train.values()],
)

end = time.time()
print(f"Global model trained in {end - start:.2f} seconds")
print("*" * 50)

Training global model
Global model trained in 10.34 seconds
**************************************************


In [12]:
mapes = {}
maes = {}

for ticker in data.keys():
    y_pred = model.predict(
        n=len(test[ticker]['close']),
        series=train[ticker]['close'],
        past_covariates=test[ticker][['high', 'low', 'volume', 'open']],
        show_warnings=False
    )
    y_test = test[ticker]['close']
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)

    mapes[ticker] = mape_
    maes[ticker] = mae_

    print(f"Global model for {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Global model for CHTR, MAPE: 2.70, MAE: 9.38
**************************************************
Global model for CMCSA, MAPE: 2.86, MAE: 1.05
**************************************************
Global model for DIS, MAPE: 3.36, MAE: 3.47
**************************************************
Global model for EA, MAPE: 2.96, MAE: 4.14
**************************************************
Global model for FOX, MAPE: 2.53, MAE: 1.15
**************************************************
Global model for FOXA, MAPE: 2.31, MAE: 1.15
**************************************************
Global model for GOOG, MAPE: 2.95, MAE: 5.17
**************************************************
Global model for GOOGL, MAPE: 3.50, MAE: 6.11
**************************************************
Global model for IPG, MAPE: 4.39, MAE: 1.19
**************************************************
Global model for LYV, MAPE: 4.06, MAE: 4.88
**************************************************
Global model for META, MAPE: 8.26, MAE: 50.0

In [13]:
np.mean(list(mapes.values())), np.mean(list(maes.values()))

(4.728725208920787, 12.19891751713104)

## Prophet model

In [14]:
models_prophet = {}

for ticker in data.keys():
    print(f"Training model for {ticker}")
    start = time.time()
    model = Prophet(
        add_encoders={
            'datetime_attribute': {
                'future': ['month', 'day', 'dayofweek', 'dayofyear', 'quarter', 'year']  # automatically generate future covariates from time series
            }
        },
        seasonality_mode='multiplicative',
    )

    model.fit(
        train[ticker]['close']
    )

    end = time.time()
    print(f"Model for {ticker} trained in {end - start:.2f} seconds")
    print("*" * 50)

    models_prophet[ticker] = model

Training model for CHTR


22:14:35 - cmdstanpy - INFO - Chain [1] start processing
22:14:38 - cmdstanpy - INFO - Chain [1] done processing


Model for CHTR trained in 3.34 seconds
**************************************************
Training model for CMCSA


22:14:38 - cmdstanpy - INFO - Chain [1] start processing
22:14:41 - cmdstanpy - INFO - Chain [1] done processing


Model for CMCSA trained in 3.22 seconds
**************************************************
Training model for DIS


...

Model for TTWO trained in 3.17 seconds
**************************************************
Training model for VZ


22:15:37 - cmdstanpy - INFO - Chain [1] start processing
22:15:39 - cmdstanpy - INFO - Chain [1] done processing


Model for VZ trained in 2.20 seconds
**************************************************
Training model for WBD


22:15:39 - cmdstanpy - INFO - Chain [1] start processing
22:15:42 - cmdstanpy - INFO - Chain [1] done processing


Model for WBD trained in 2.75 seconds
**************************************************


In [15]:
mapes_prophet = {}
maes_prophet = {}

for ticker in data.keys():
    y_test = test[ticker]['close']
    y_pred = models_prophet[ticker].predict(
        n=len(y_test),
        show_warnings=False
    )
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)
    mapes_prophet[ticker] = mape_
    maes_prophet[ticker] = mae_
    print(f"Ticker: {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Ticker: CHTR, MAPE: 19.13, MAE: 66.45
**************************************************
Ticker: CMCSA, MAPE: 18.90, MAE: 6.96
**************************************************
Ticker: DIS, MAPE: 11.93, MAE: 12.41
**************************************************
Ticker: EA, MAPE: 6.08, MAE: 8.87
**************************************************
Ticker: FOX, MAPE: 29.66, MAE: 13.88
**************************************************
Ticker: FOXA, MAPE: 28.44, MAE: 14.18
**************************************************
Ticker: GOOG, MAPE: 8.72, MAE: 15.13
**************************************************
Ticker: GOOGL, MAPE: 8.91, MAE: 15.33
**************************************************
Ticker: IPG, MAPE: 49.38, MAE: 13.89
**************************************************
Ticker: LYV, MAPE: 17.00, MAE: 21.60
**************************************************
Ticker: META, MAPE: 9.00, MAE: 49.77
**************************************************
Ticker: MTCH, MAPE: 23.76, MAE:

In [16]:
np.mean(list(mapes_prophet.values())), np.mean(list(maes_prophet.values()))

(21.720190647304413, 25.362521014161793)

## Theta model

In [17]:
models_theta = {}

for ticker in data.keys():
    print(f"Training model for {ticker}")
    start = time.time()
    model = AutoTheta(
        season_length=30
    )

    model.fit(
        series=train[ticker]['close']
    )

    end = time.time()
    print(f"Model for {ticker} trained in {end - start:.2f} seconds")
    print("*" * 50)

    models_theta[ticker] = model

Training model for CHTR
Model for CHTR trained in 0.66 seconds
**************************************************
Training model for CMCSA
Model for CMCSA trained in 1.02 seconds
**************************************************
Training model for DIS
Model for DIS trained in 0.68 seconds
**************************************************
Training model for EA
Model for EA trained in 0.73 seconds
**************************************************
Training model for FOX
Model for FOX trained in 0.27 seconds
**************************************************
Training model for FOXA
Model for FOXA trained in 0.27 seconds
**************************************************
Training model for GOOG
Model for GOOG trained in 0.99 seconds
**************************************************
Training model for GOOGL
Model for GOOGL trained in 1.05 seconds
**************************************************
Training model for IPG
Model for IPG trained in 0.57 seconds
*******************************

In [18]:
mapes_theta = {}
maes_theta = {}

for ticker in data.keys():
    y_test = test[ticker]['close']
    y_pred = models_theta[ticker].predict(
        n=len(y_test),
        show_warnings=False
    )
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)
    mapes_theta[ticker] = mape_
    maes_theta[ticker] = mae_
    print(f"Ticker: {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Ticker: CHTR, MAPE: 20.73, MAE: 73.41
**************************************************
Ticker: CMCSA, MAPE: 5.59, MAE: 2.12
**************************************************
Ticker: DIS, MAPE: 13.36, MAE: 12.49
**************************************************
Ticker: EA, MAPE: 8.75, MAE: 12.90
**************************************************
Ticker: FOX, MAPE: 17.40, MAE: 8.28
**************************************************
Ticker: FOXA, MAPE: 16.65, MAE: 8.46
**************************************************
Ticker: GOOG, MAPE: 6.28, MAE: 11.15
**************************************************
Ticker: GOOGL, MAPE: 6.38, MAE: 11.23
**************************************************
Ticker: IPG, MAPE: 7.12, MAE: 1.91
**************************************************
Ticker: LYV, MAPE: 19.22, MAE: 24.72
**************************************************
Ticker: META, MAPE: 19.24, MAE: 113.75
**************************************************
Ticker: MTCH, MAPE: 6.92, MAE: 2.

In [19]:
np.mean(list(mapes_theta.values())), np.mean(list(maes_theta.values()))

(14.238836069726121, 27.631980046268737)

## Linear Regression

### Local model

In [20]:
models_lr = {}

for ticker in data.keys():
    print(f"Training model for {ticker}")
    start = time.time()
    model = LinearRegressionModel(
        lags=15,
        lags_past_covariates=1,
        lags_future_covariates=[0],
        output_chunk_length=1,
        random_state=42,
        add_encoders={
            'datetime_attribute': {
                'future': ['month', 'day', 'dayofweek', 'dayofyear', 'quarter', 'year'] # automatically generate future covariates from time series
            }
        }
    )

    model.fit(
        train[ticker]['close'],
        past_covariates=train[ticker][['high', 'low', 'volume', 'open']],
    )

    end = time.time()
    print(f"Model for {ticker} trained in {end - start:.2f} seconds")
    print("*" * 50)

    models_lr[ticker] = model

Training model for CHTR
Model for CHTR trained in 0.13 seconds
**************************************************
Training model for CMCSA
Model for CMCSA trained in 0.13 seconds
**************************************************
Training model for DIS
Model for DIS trained in 0.12 seconds
**************************************************
Training model for EA
Model for EA trained in 0.12 seconds
**************************************************
Training model for FOX
Model for FOX trained in 0.10 seconds
**************************************************
Training model for FOXA
Model for FOXA trained in 0.09 seconds
**************************************************
Training model for GOOG
Model for GOOG trained in 0.12 seconds
**************************************************
Training model for GOOGL
Model for GOOGL trained in 0.12 seconds
**************************************************
Training model for IPG
Model for IPG trained in 0.12 seconds
*******************************

In [21]:
mapes_lr = {}
maes_lr = {}

for ticker in data.keys():
    y_test = test[ticker]['close']
    y_pred = models_lr[ticker].predict(
        n=len(y_test),
        past_covariates=test[ticker][['high', 'low', 'volume', 'open']],
        show_warnings=False
    )
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)
    mapes_lr[ticker] = mape_
    maes_lr[ticker] = mae_
    print(f"Ticker: {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Ticker: CHTR, MAPE: 2.36, MAE: 8.22
**************************************************
Ticker: CMCSA, MAPE: 3.72, MAE: 1.37
**************************************************
Ticker: DIS, MAPE: 1870.68, MAE: 1805.00
**************************************************
Ticker: EA, MAPE: 1.98, MAE: 2.79
**************************************************
Ticker: FOX, MAPE: 596.26, MAE: 282.68
**************************************************
Ticker: FOXA, MAPE: 502080.58, MAE: 250106.02
**************************************************
Ticker: GOOG, MAPE: 2.83, MAE: 4.91
**************************************************
Ticker: GOOGL, MAPE: 2.78, MAE: 4.77
**************************************************
Ticker: IPG, MAPE: 3.02, MAE: 0.83
**************************************************
Ticker: LYV, MAPE: 3.31, MAE: 3.98
**************************************************
Ticker: META, MAPE: 3.08, MAE: 17.46
**************************************************
Ticker: MTCH, MAPE: 326.51

In [22]:
np.mean(list(mapes_lr.values())), np.mean(list(maes_lr.values()))

(21955.389564011555, 10974.178714378988)

### Global model

In [23]:
print(f"Training global model")
start = time.time()
model = LinearRegressionModel(
    lags=15,
    lags_past_covariates=1,
    lags_future_covariates=[0],
    output_chunk_length=1,
    random_state=42,
    add_encoders={
        'datetime_attribute': {
            'future': ['month', 'day', 'dayofweek', 'dayofyear', 'quarter', 'year'] # automatically generate future covariates from time series
        }
    }
)

model.fit(
    [ts['close'] for ts in train.values()],
    past_covariates=[ts[['high', 'low', 'volume', 'open']] for ts in train.values()],
)

end = time.time()
print(f"Global model trained in {end - start:.2f} seconds")
print("*" * 50)

Training global model
Global model trained in 2.76 seconds
**************************************************


In [24]:
mapes_lr_global = {}
maes_lr_global = {}

for ticker in data.keys():
    y_pred = model.predict(
        n=len(test[ticker]['close']),
        series=train[ticker]['close'],
        past_covariates=test[ticker][['high', 'low', 'volume', 'open']],
        show_warnings=False
    )
    y_test = test[ticker]['close']
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)

    mapes_lr_global[ticker] = mape_
    maes_lr_global[ticker] = mae_

    print(f"Global model for {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Global model for CHTR, MAPE: 2.59, MAE: 9.03
**************************************************
Global model for CMCSA, MAPE: 1.69, MAE: 0.63
**************************************************
Global model for DIS, MAPE: 2.21, MAE: 2.18
**************************************************
Global model for EA, MAPE: 1.96, MAE: 2.74
**************************************************
Global model for FOX, MAPE: 2.17, MAE: 1.00
**************************************************
Global model for FOXA, MAPE: 2.17, MAE: 1.07
**************************************************
Global model for GOOG, MAPE: 2.22, MAE: 3.84
**************************************************
Global model for GOOGL, MAPE: 2.24, MAE: 3.84
**************************************************
Global model for IPG, MAPE: 1.87, MAE: 0.52
**************************************************
Global model for LYV, MAPE: 2.56, MAE: 3.01
**************************************************
Global model for META, MAPE: 2.80, MAE: 15.7

In [25]:
np.mean(list(mapes_lr_global.values())), np.mean(list(maes_lr_global.values()))

(2.273185437737936, 3.593161854991359)

## ARIMA

In [26]:
models_arima = {}

for ticker in data.keys():
    print(f"Training model for {ticker}")
    start = time.time()
    model = AutoARIMA()

    model.fit(
        series=train[ticker]['close'],
    )

    end = time.time()
    print(f"Model for {ticker} trained in {end - start:.2f} seconds")
    print("*" * 50)

    models_arima[ticker] = model

Training model for CHTR
Model for CHTR trained in 0.20 seconds
**************************************************
Training model for CMCSA
Model for CMCSA trained in 0.43 seconds
**************************************************
Training model for DIS
Model for DIS trained in 0.42 seconds
**************************************************
Training model for EA
Model for EA trained in 0.20 seconds
**************************************************
Training model for FOX
Model for FOX trained in 0.15 seconds
**************************************************
Training model for FOXA
Model for FOXA trained in 0.14 seconds
**************************************************
Training model for GOOG
Model for GOOG trained in 0.51 seconds
**************************************************
Training model for GOOGL
Model for GOOGL trained in 0.44 seconds
**************************************************
Training model for IPG
Model for IPG trained in 1.31 seconds
*******************************

In [27]:
mapes_arima = {}
maes_arima = {}

for ticker in data.keys():
    y_test = test[ticker]['close']
    y_pred = models_arima[ticker].predict(
        n=len(y_test),
        show_warnings=False
    )
    mape_ = mape(y_test, y_pred)
    mae_ = mae(y_test, y_pred)
    mapes_arima[ticker] = mape_
    maes_arima[ticker] = mae_
    print(f"Ticker: {ticker}, MAPE: {mape_:.2f}, MAE: {mae_:.2f}")
    print("*" * 50)

Ticker: CHTR, MAPE: 22.05, MAE: 78.03
**************************************************
Ticker: CMCSA, MAPE: 5.19, MAE: 2.00
**************************************************
Ticker: DIS, MAPE: 12.90, MAE: 12.06
**************************************************
Ticker: EA, MAPE: 10.50, MAE: 15.52
**************************************************
Ticker: FOX, MAPE: 17.53, MAE: 8.34
**************************************************
Ticker: FOXA, MAPE: 16.86, MAE: 8.56
**************************************************
Ticker: GOOG, MAPE: 6.76, MAE: 11.73
**************************************************
Ticker: GOOGL, MAPE: 6.85, MAE: 11.78
**************************************************
Ticker: IPG, MAPE: 5.77, MAE: 1.56
**************************************************
Ticker: LYV, MAPE: 22.01, MAE: 28.16
**************************************************
Ticker: META, MAPE: 17.01, MAE: 100.74
**************************************************
Ticker: MTCH, MAPE: 7.21, MAE: 2

In [28]:
np.mean(list(mapes_arima.values())), np.mean(list(maes_arima.values()))

(15.321454476582323, 28.68816789027297)